# CitibikePy

---

## Code to clean and reshape citibike data for consumption by tableau

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import math
import requests
from datetime import datetime
import json
# Import the geodesic module from geopy library 
from geopy import distance

## Fetch the citibike station data

In [2]:
# Set the API base URL
api_url = "https://gbfs.citibikenyc.com/gbfs/en/station_information.json"


# Define an empty list to fetch the weather data for each city
station_data = []

# Run an API request for all of the citibike stations
station_response = requests.get(api_url)
station_json = station_response.json()
station_data = station_json['data']['stations']

In [3]:
# Convert the citibike station data into a Pandas DataFrame
station_data_df = pd.DataFrame(station_data)
station_data_reduced = station_data_df[["name", "capacity", "lat", "lon"]].copy()
station_data_reduced.rename(columns = {'lat':'latitude', 'lon':'longitude'}, inplace = True)


In [4]:
station_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1954 entries, 0 to 1953
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   external_id                     1954 non-null   object 
 1   has_kiosk                       1954 non-null   bool   
 2   rental_methods                  1954 non-null   object 
 3   station_type                    1954 non-null   object 
 4   legacy_id                       1954 non-null   object 
 5   eightd_has_key_dispenser        1954 non-null   bool   
 6   lat                             1954 non-null   float64
 7   lon                             1954 non-null   float64
 8   station_id                      1954 non-null   object 
 9   electric_bike_surcharge_waiver  1954 non-null   bool   
 10  capacity                        1954 non-null   int64  
 11  short_name                      1954 non-null   object 
 12  rental_uris                     19

## Fetch the citibike usage data by month for the last 12 month period
### Bike usage (trip) data for 1 year is very large to pull into tableau. Consoluate the information by generating statistics.

In [5]:
# Read ride data for the past year, June 2022 - May 2023
month_names = ['Jun 2022','Jul 2022','Aug 2022','Sep 2022','Oct 2022','Nov 2022','Dec 2022','Jan 2023','Feb 2023','Mar 2023','Apr 2023','May 2023']
month_numbers = ['06','07','08','09','10','11','12','01','02','03','04','05']
station_stats = station_data_reduced
station_stats_12month = pd.DataFrame()
ride_stats_12month = pd.DataFrame()

for i in range(0,12):
    if i in range(0,7):
        filename = 'data/JC-2022' + month_numbers[i] + '-citibike-tripdata.csv'
    else:
        filename = 'data/JC-2023' + month_numbers[i] + '-citibike-tripdata.csv'
    # read in monthly file
    monthly_rides_df = pd.read_csv(filename)
    print(f"generating month {month_names[i]}")
    
    # clean data
    # remove rides rows with ended_at, end_lat or end_lng missing
    monthly_rides_df.dropna(subset = ['ended_at', 'end_lat', 'end_lng'], inplace=True)
    monthly_rides_df['end_station_name'] =  monthly_rides_df['end_station_name'].fillna('Missing')
    monthly_rides_df['month'] = np.full(len(monthly_rides_df.index), month_names[i])
    
    # caculate ride time and ride distance
    ride_times = []
    ride_distances = []
    for index, row in monthly_rides_df.iterrows():
        start_time = datetime.strptime(row['started_at'], '%Y-%m-%d %H:%M:%S')
        end_time = datetime.strptime(row['ended_at'], '%Y-%m-%d %H:%M:%S')
        delta =  end_time - start_time
        ride_times.append(round(delta.total_seconds() / 60,1))
        ride_distances.append(distance.distance((row['start_lat'], row['start_lng']), (row['end_lat'], row['end_lng'])).miles)
    monthly_rides_df['ride_time'] = ride_times
    monthly_rides_df['ride_distance'] = ride_distances
    
    # split started_at and ended_at into date and time columns
    starts = monthly_rides_df["started_at"].str.split(" ", n = 1, expand = True)
    start_hour = starts[1].str.split(":", expand = True)[0]
    ends = monthly_rides_df["ended_at"].str.split(" ", n = 1, expand = True)
    end_hour = ends[1].str.split(":", expand = True)[0]
    # making new columns from separated data
    monthly_rides_df['start_date'] = starts[0]
    monthly_rides_df['start_hour'] = start_hour
    monthly_rides_df['end_date'] = ends[0]
    monthly_rides_df['end_hour'] = end_hour
    
    # Dropping old columns
    monthly_rides_df.drop(columns =["started_at","ended_at"], inplace = True)
    
    # generate monthly ride statistics
    monthly_ride_total = len(monthly_rides_df.index)
    not_returned_df = monthly_rides_df[monthly_rides_df['end_station_name'] == 'Missing']
    not_returned = len(not_returned_df.index)
    percent_not_returned = not_returned / monthly_ride_total
    avg_ride_distance = monthly_rides_df['ride_distance'].mean()
    avg_ride_time = monthly_rides_df['ride_time'].mean()
    percent_riders_member = len(monthly_rides_df[monthly_rides_df['member_casual'] == 'member'].index) / monthly_ride_total
    percent_riders_casual = len(monthly_rides_df[monthly_rides_df['member_casual'] == 'casual'].index) / monthly_ride_total
    percent_notreturned_casual = len(not_returned_df[not_returned_df['member_casual'] == 'casual'].index) / not_returned
    data_values = [[month_names[i], monthly_ride_total, not_returned, percent_not_returned, avg_ride_distance, avg_ride_time, percent_riders_member, percent_riders_casual, percent_notreturned_casual]]
    column_names = ['month','monthly_ride_total','not_returned','percent_not_returned','avg_ride_distance_mi','avg_ride_time_min','percent_riders_member','percent_riders_casual','percent_notreturned_casual']
    ride_stats_monthly = pd.DataFrame(data=data_values,columns=column_names) 
    
    # append the monthly ride records to the ride records for the last 12 months
    ride_stats_12month = pd.concat([ride_stats_12month, ride_stats_monthly], ignore_index=True)
    
    # generate monthly station statistics
    start_count = pd.DataFrame(monthly_rides_df['start_station_name'].value_counts()).reset_index()
    start_count.columns = ['name', 'start_count']
    start_count['month'] = np.full(len(start_count.index), month_names[i])
    end_count = pd.DataFrame(monthly_rides_df['end_station_name'].value_counts()).reset_index()
    end_count.columns = ['name','end_count']
    
    end_count['month'] = np.full(len(end_count.index), month_names[i])
    month_stats = pd.merge(start_count, end_count, on=['name','month'], how='outer')
    # clean NaN/null
    month_stats.fillna(0, inplace=True)
    month_stats["start_count"] = month_stats["start_count"].astype(int)
    month_stats["end_count"] = month_stats["end_count"].astype(int)
    
    # add the latitude and longitude of each station
    lng_list = []
    lat_list = []
    for index, row in month_stats.iterrows():
        station_name = row['name'].strip().replace('\t', ' ').replace('\\t', ' ').replace('\n', '')
        if station_name == 'Missing':
            lng_list.append(float("NaN"))
            lat_list.append(float("NaN"))
            continue
        # fix up known station name discrepancies
        if station_name == 'Columbus Dr at Exchange Pl':
            station_name = 'Exchange'
        if station_name == 'Pier 40 Dock Station':
            station_name = 'Pier 40'
        if station_name == 'Grove St - PATH':
            station_name = 'Grove St PATH'
        if station_name == 'W 40 St & 5 Ave':
            station_name = '40 St & 5 Ave'
        if station_name == 'W 34 St &\tHudson Blvd E':
            station_name = 'W 34 St & Hudson Blvd E'
        if station_name == 'Broadway\t& W 48 St':
            station_name = 'Broadway & W 48 St'
        if station_name == 'Murray St\t& West St':
            station_name = 'Murray St & West St'
        
        found_row = station_stats.loc[station_stats["name"].str.contains(station_name)]
        # check if the search for the station failed
        if found_row.empty:
            print(f"WARNING: Could not find station {station_name}. Skipping latitude/longitude for entry.")
            lng_list.append(float("NaN"))
            lat_list.append(float("NaN"))
            continue

        # if the station search succeeded, add the longitude and latitude to the lists
        lon = found_row.iloc[0]['longitude']
        lat = found_row.iloc[0]['latitude']
        lng_list.append(lon)
        lat_list.append(lat)
    month_stats['latitude'] = lat_list
    month_stats['longitude'] = lng_list
    
    
    # reorder the columns and add this month's trip stats to the running monthly trip stats
    station_stats_12month = pd.concat([station_stats_12month, month_stats.loc[:,['name','latitude', 'longitude', 'month','start_count','end_count']]], ignore_index=True)
    


generating month Jun 2022
generating month Jul 2022
generating month Aug 2022
generating month Sep 2022
generating month Oct 2022
generating month Nov 2022
generating month Dec 2022
generating month Jan 2023
generating month Feb 2023
generating month Mar 2023
generating month Apr 2023
generating month May 2023


In [6]:
station_stats_12month.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1691 entries, 0 to 1690
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         1691 non-null   object 
 1   latitude     1674 non-null   float64
 2   longitude    1674 non-null   float64
 3   month        1691 non-null   object 
 4   start_count  1691 non-null   int64  
 5   end_count    1691 non-null   int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 79.4+ KB


In [7]:
# find the top ten stations by monthly average for citibike pickup
top10_start_stations = pd.DataFrame(station_stats_12month.groupby(["name"]).mean()["start_count"].sort_values(ascending=False).reset_index()[0:10])
top10_start_stations["start_count"] = top10_start_stations["start_count"].map("{:.2f}".format)
top10_start_stations.rename(columns = {'start_count':'mean_monthly_pickups'}, inplace = True)
lat_list = []
long_list = []

# add the longitude and latitude to the top10 list
for index, row in top10_start_stations.iterrows():
    station_name = row['name']
    lon = station_stats.loc[station_stats["name"].str.contains(station_name)].iloc[0]['longitude']
    lat = station_stats.loc[station_stats["name"].str.contains(station_name)].iloc[0]['latitude']
    long_list.append(lon)
    lat_list.append(lat)
top10_start_stations['latitude'] = lat_list
top10_start_stations['longitude'] = long_list

# find the top ten stations by monthly average for citibike dropoff
top10_end_stations = pd.DataFrame(station_stats_12month.groupby(["name"]).mean()["end_count"].sort_values(ascending=False).reset_index()[0:10])
top10_end_stations["end_count"] = top10_end_stations["end_count"].map("{:.2f}".format)
top10_end_stations.rename(columns = {'end_count':'mean_monthly_dropoffs'}, inplace = True)
lat_list = []
long_list = []

for index, row in top10_end_stations.iterrows():
    station_name = row['name']
    lon = station_stats.loc[station_stats["name"].str.contains(station_name)].iloc[0]['longitude']
    lat = station_stats.loc[station_stats["name"].str.contains(station_name)].iloc[0]['latitude']
    long_list.append(lon)
    lat_list.append(lat)
top10_end_stations['latitude'] = lat_list
top10_end_stations['longitude'] = long_list

In [8]:
top10_start_stations

,name,mean_monthly_pickups,latitude,longitude
0,Grove St PATH,3941.75,40.719410,-74.043090
1,Hoboken Terminal - River St & Hudson Pl,3592.50,40.736068,-74.029127
2,South Waterfront Walkway - Sinatra Dr & 1 St,3003.50,40.736982,-74.027781
3,Hoboken Terminal - Hudson St & Hudson Pl,2510.92,40.735938,-74.030305
4,City Hall - Washington St & 1 St,2149.92,40.737360,-74.030970
5,Newport Pkwy,1900.33,40.728745,-74.032108
6,Newport PATH,1810.25,40.727224,-74.033759
7,Hamilton Park,1807.75,40.727596,-74.044247
8,Sip Ave,1756.00,40.731009,-74.064437
9,Hoboken Ave at Monmouth St,1686.67,40.735208,-74.046964


In [9]:
top10_end_stations

,name,mean_monthly_dropoffs,latitude,longitude
0,Grove St PATH,4178.00,40.719410,-74.043090
1,Hoboken Terminal - River St & Hudson Pl,3547.83,40.736068,-74.029127
2,South Waterfront Walkway - Sinatra Dr & 1 St,3022.25,40.736982,-74.027781
3,Hoboken Terminal - Hudson St & Hudson Pl,2514.50,40.735938,-74.030305
4,City Hall - Washington St & 1 St,2162.50,40.737360,-74.030970
5,Newport Pkwy,1899.17,40.728745,-74.032108
6,Hamilton Park,1818.58,40.727596,-74.044247
7,Newport PATH,1816.17,40.727224,-74.033759
8,Hoboken Ave at Monmouth St,1740.25,40.735208,-74.046964
9,Sip Ave,1737.00,40.731009,-74.064437


In [10]:
station_stats_12month



,name,latitude,longitude,month,start_count,end_count
0,South Waterfront Walkway - Sinatra Dr & 1 St,40.736982,-74.027781,Jun 2022,4602,4647
1,Grove St PATH,40.719410,-74.043090,Jun 2022,4482,4767
2,Hoboken Terminal - Hudson St & Hudson Pl,40.735938,-74.030305,Jun 2022,3463,3407
3,Hoboken Terminal - River St & Hudson Pl,40.736068,-74.029127,Jun 2022,3074,2994
4,Newport Pkwy,40.728745,-74.032108,Jun 2022,2784,2787
...,...,...,...,...,...,...
1686,W 11 St & 6 Ave,40.735324,-73.998004,May 2023,0,1
1687,Broadway & W 142 St,40.824686,-73.951947,May 2023,0,1
1688,W 37 St & 10 Ave,40.756604,-73.997901,May 2023,0,1
1689,Spruce St & Nassau St,40.711464,-74.005524,May 2023,0,1


In [11]:
# save to a csv files
station_stats.to_csv('data/stations.csv', index=False)
ride_stats_12month.to_csv('data/ridestats1yr.csv', index=False)
station_stats_12month.to_csv('data/stationstats1yr.csv', index=False)
top10_start_stations.to_csv('data/top10starts.csv', index=False)
top10_end_stations.to_csv('data/top10ends.csv', index=False)